In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
from tqdm import tqdm  # for progress bars

class PutOptionOptimizer:
    def __init__(self):
        # Mock data - replace with your Bloomberg data loading
        np.random.seed(42)
        dates = pd.date_range('2010-01-01', '2023-12-31')
        prices = np.cumprod(1 + np.random.normal(0.0003, 0.01, len(dates)))
        self.data = pd.DataFrame({'price': prices}, index=dates)
        self.data['returns'] = np.log(self.data['price']/self.data['price'].shift(1))
        self.data['volatility'] = self.data['returns'].rolling(21).std() * np.sqrt(252)
        
    def generate_signals(self, ma_window=50, vol_threshold=0.2):
        """Generate signals using moving average and volatility filters"""
        self.data['ma'] = self.data['price'].rolling(ma_window).mean()
        self.data['signal'] = (
            (self.data['price'] > self.data['ma']) & 
            (self.data['volatility'] > vol_threshold)
        ).astype(int)
        return self.data['signal']
    
    def simulate_puts(self, strike_pct=0.85, maturity_days=63):
        """Simulate put option performance"""
        results = []
        signal_dates = self.data[self.data['signal']==1].index
        
        for entry_date in signal_dates:
            entry_idx = self.data.index.get_loc(entry_date)
            if entry_idx + maturity_days >= len(self.data):
                continue
                
            entry_price = self.data.loc[entry_date, 'price']
            expiry_date = self.data.index[entry_idx + maturity_days]
            expiry_price = self.data.loc[expiry_date, 'price']
            strike_price = entry_price * strike_pct
            
            # Calculate daily PnL path (simplified)
            mtm_prices = self.data.loc[entry_date:expiry_date, 'price']
            intrinsic_values = np.maximum(strike_price - mtm_prices, 0)
            daily_pnl = intrinsic_values.diff()
            
            results.append({
                'entry_date': entry_date,
                'expiry_date': expiry_date,
                'strike_pct': strike_pct,
                'final_pnl': intrinsic_values[-1],
                'total_pnl': intrinsic_values.sum(),
                'max_drawdown': (intrinsic_values.max() - intrinsic_values.min()),
                'sharpe_ratio': daily_pnl.mean() / daily_pnl.std() if daily_pnl.std() > 0 else 0
            })
        
        return pd.DataFrame(results)
    
    def evaluate_strategy(self, params):
        """Evaluate strategy with given parameters"""
        ma_window, strike_pct, vol_threshold = params
        ma_window = int(max(20, min(200, ma_window)))
        
        signals = self.generate_signals(ma_window, vol_threshold)
        trades = self.simulate_puts(strike_pct)
        
        if trades.empty:
            return -np.inf  # Penalize no-trade scenarios
        
        # Combined score (60% final PnL, 20% Sharpe, 20% drawdown)
        score = (
            0.6 * trades['final_pnl'].mean() +
            0.2 * trades['sharpe_ratio'].mean() -
            0.2 * trades['max_drawdown'].mean()
        )
        return score
    
    def grid_search_optimize(self):
        """Grid search optimization over key parameters"""
        param_grid = {
            'ma_window': np.arange(20, 201, 20),  # 20 to 200 days
            'strike_pct': np.linspace(0.75, 0.9, 7),  # 75% to 90% strikes
            'vol_threshold': np.linspace(0.15, 0.35, 5)  # Volatility thresholds
        }
        
        best_score = -np.inf
        best_params = {}
        results = []
        
        # Create progress bar
        total_combinations = (
            len(param_grid['ma_window']) * 
            len(param_grid['strike_pct']) * 
            len(param_grid['vol_threshold'])
        )
        
        with tqdm(total=total_combinations, desc="Optimizing") as pbar:
            for ma in param_grid['ma_window']:
                for strike in param_grid['strike_pct']:
                    for vol in param_grid['vol_threshold']:
                        score = self.evaluate_strategy((ma, strike, vol))
                        results.append({
                            'ma_window': ma,
                            'strike_pct': strike,
                            'vol_threshold': vol,
                            'score': score
                        })
                        
                        if score > best_score:
                            best_score = score
                            best_params = {
                                'ma_window': ma,
                                'strike_pct': strike,
                                'vol_threshold': vol
                            }
                        pbar.update(1)
        
        self.results = pd.DataFrame(results)
        self.best_params = best_params
        return best_params
    
    def plot_optimization(self):
        """Visualize optimization results"""
        if self.results is None:
            raise ValueError("Run grid_search_optimize() first")
            
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # MA Window vs Strike %
        pivot1 = self.results.pivot_table(index='ma_window', columns='strike_pct', values='score')
        axes[0].imshow(pivot1, aspect='auto', cmap='viridis')
        axes[0].set_title("MA Window vs Strike %")
        axes[0].set_xlabel("Strike Percentage")
        axes[0].set_ylabel("MA Window (days)")
        
        # MA Window vs Vol Threshold
        pivot2 = self.results.pivot_table(index='ma_window', columns='vol_threshold', values='score')
        axes[1].imshow(pivot2, aspect='auto', cmap='viridis')
        axes[1].set_title("MA Window vs Vol Threshold")
        axes[1].set_xlabel("Volatility Threshold")
        axes[1].set_ylabel("MA Window (days)")
        
        # Strike % vs Vol Threshold
        pivot3 = self.results.pivot_table(index='strike_pct', columns='vol_threshold', values='score')
        axes[2].imshow(pivot3, aspect='auto', cmap='viridis')
        axes[2].set_title("Strike % vs Vol Threshold")
        axes[2].set_xlabel("Volatility Threshold")
        axes[2].set_ylabel("Strike Percentage")
        
        plt.tight_layout()
        plt.show()

# Example Usage
if __name__ == "__main__":
    optimizer = PutOptionOptimizer()
    
    print("Running grid search optimization...")
    best_params = optimizer.grid_search_optimize()
    
    print("\nBest Parameters Found:")
    for param, value in best_params.items():
        print(f"{param:>15}: {value:.2f}")
    
    print("\nVisualizing optimization results...")
    optimizer.plot_optimization()
    
    # Backtest with best parameters
    print("\nRunning backtest with optimized parameters...")
    optimizer.generate_signals(best_params['ma_window'], best_params['vol_threshold'])
    trades = optimizer.simulate_puts(best_params['strike_pct'])
    
    print("\nBacktest Results Summary:")
    print(trades.describe().to_string())
    
    # Plot cumulative PnL
    plt.figure(figsize=(12, 6))
    trades.set_index('entry_date')['final_pnl'].cumsum().plot()
    plt.title("Cumulative PnL of Optimized Strategy")
    plt.ylabel("Cumulative PnL")
    plt.grid()
    plt.show()